In [4]:
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import descartes
import sys
import sklearn
from datetime import datetime as dt

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#  Merge and Map Demographics into a Final DF

1. First, calculate per county population density
2. Next, align dates so all dataframes go back through 1990
3. Merge all demogs dfs into a master df
4. Save the cleaned data

In [12]:
#Import the data
#Import the county to grid mapping, demogs, hhi, unemployment, and arson crimes
gitdir = '/Users/jades/1001 Intro to Data Science Notebooks/Project/wildfires-1001/'
#gitdir = '/Users/saraprice/Documents/NYU/Fall_2020/DS_GA_1001/wildfires-1001/'
subdir_from = 'data/clean_data/mapping_tables/'
filename_from = 'CA_counties_clean.csv'

df_area = pd.read_csv(gitdir + subdir_from + filename_from)

subdir_from = 'data/clean_data/county_grid/'
filename_from = 'county_grid.shp'

df_map = gpd.read_file(gitdir + subdir_from + filename_from)

subdir_from = 'data/clean_data/ca_demogs/'
filename_from = 'ca_demogs_clean.csv'

df_demogs = pd.read_csv(gitdir + subdir_from + filename_from)

subdir_from = 'data/clean_data/ca_demogs/'
filename_from = 'median_hhi_clean.csv'

df_hhi = pd.read_csv(gitdir + subdir_from + filename_from)

subdir_from = 'data/clean_data/ca_demogs/'
filename_from = 'unemployment_clean.csv'

df_unemployment = pd.read_csv(gitdir + subdir_from + filename_from)

subdir_from = 'data/clean_data/arson_crimes/'
filename_from = 'arson_clean.csv'

df_arson = pd.read_csv(gitdir + subdir_from + filename_from)

In [13]:
#Confirm the data is what we expect
print('County Area')
print(df_area.head())
print('-----------------------------------------------------------------')
print('Mapping File')
print(df_map.head())
print('-----------------------------------------------------------------')
print('Demogs')
print(df_demogs.head())
print('-----------------------------------------------------------------')
print('Household Income')
print(df_hhi.head())
print('-----------------------------------------------------------------')
print('Unemployment')
print(df_unemployment.head())
print('-----------------------------------------------------------------')
print('Arson')
print(df_arson.head())
print('-----------------------------------------------------------------')

County Area
            NAME  COUNTYFP  GEOID       ALAND      AWATER                                           geometry   COUNTY_AREA
0         Sierra        91   6091  2468694587    23299110  POLYGON ((-13431319.75097945 4821511.426395644...  4.200450e+09
1     Sacramento        67   6067  2499183617    76073827  POLYGON ((-13490651.47641084 4680831.603393857...  4.205267e+09
2  Santa Barbara        83   6083  7084000598  2729814515  MULTIPOLYGON (((-13423116.77225655 4042044.148...  1.449841e+10
3      Calaveras         9   6009  2641820834    43806026  POLYGON ((-13428575.483353 4627725.227535474, ...  4.356213e+09
4        Ventura       111   6111  4773390489   945942791  MULTIPOLYGON (((-13317853.59433417 3931602.413...  8.413293e+09
-----------------------------------------------------------------
Mapping File
           NAME COUNTYFP  GEOID       ALAND     AWATER    COUNTY_ARE     GRID_AREA  GRID_ID    COUNTY_GRI  COUNTY_G_1                                           geometry
0 

In [14]:
df_area.shape

(58, 7)

In [15]:
#First calculate per county population density

#Merge demogs with county area
df_demogs = df_demogs.merge(df_area[['COUNTYFP', 'COUNTY_AREA']], how='left', left_on='FIPS', right_on='COUNTYFP') 

#Calculate the density normalized features in km squared
df_demogs['POPDENSITY'] = df_demogs['POPESTIMATE']/df_demogs['COUNTY_AREA']*1000000
df_demogs['POPDENSITY_MALE'] = df_demogs['POPEST_MALE']/df_demogs['COUNTY_AREA']*1000000
df_demogs['POPDENSITY_FEM'] = df_demogs['POPEST_FEM']/df_demogs['COUNTY_AREA']*1000000
df_demogs['AGEUNDER13_TOT'] = df_demogs['AGEUNDER13_TOT']/df_demogs['COUNTY_AREA']*1000000
df_demogs['AGE1424_TOT'] = df_demogs['AGE1424_TOT']/df_demogs['COUNTY_AREA']*1000000
df_demogs['AGE2544_TOT'] = df_demogs['AGE2544_TOT']/df_demogs['COUNTY_AREA']*1000000
df_demogs['AGE4564_TOT'] = df_demogs['AGE4564_TOT']/df_demogs['COUNTY_AREA']*1000000
df_demogs['AGE65PLUS_TOT'] = df_demogs['AGE65PLUS_TOT']/df_demogs['COUNTY_AREA']*1000000

df_demogs = df_demogs.drop(['POPESTIMATE', 'POPEST_MALE', 'POPEST_FEM', 'COUNTYFP'], axis=1)

In [16]:
#Next, align dates so all dataframes go back through 1990
#We trim the arson data since it goes back through 1985
#We extrapolate backwards the last year of data for ca demogs and unemployment

df_arson = df_arson[df_arson['Year'] >= 1990]

for i in range(18): #Duplicate 2008 onto 1990 to 2007
    df_demogs = df_demogs.append(df_demogs[df_demogs['YEAR'] == 2008].assign(YEAR=i+1990))
    
for i in range(21): #Duplicate 2011 onto 1990 to 2010
    df_unemployment = df_unemployment.append(df_unemployment[df_unemployment['year'] == 2011].assign(year=i+1990))

In [19]:
df_map['COUNTYFP'] = df_map['COUNTYFP'].astype(int)
df_map['GRID_ID'] = df_map['GRID_ID'].astype(int)

In [20]:
#Finally, create the master dataframe and merge

df_master = df_map[['NAME', 'COUNTYFP', 'GRID_ID']].merge(df_demogs, how='left', left_on='COUNTYFP', right_on='FIPS')
df_master = df_master.drop(['FIPS', 'County Name'], axis=1)

df_master = df_master.merge(df_arson, how='left', left_on=('COUNTYFP', 'YEAR'), right_on=('FIPS', 'Year'))
df_master = df_master.drop(['FIPS', 'County Name', 'Year'], axis=1)

df_master = df_master.merge(df_unemployment, how='left', left_on=('COUNTYFP', 'YEAR'), right_on=('FIPS', 'year'))
df_master = df_master.drop(['FIPS', 'County Name', 'year'], axis=1)

df_master = df_master.merge(df_hhi, how='left', left_on=('COUNTYFP'), right_on=('FIPS'))
df_master = df_master.drop(['FIPS', 'County Name'], axis=1)

In [23]:
len(df_master['GRID_ID'].unique())

134

In [24]:
#Save the cleaned dataframe

subdir_to = 'data/clean_data/ca_demogs/'
filename_to = 'demogs_arson_master.csv'
df_master.to_csv(gitdir + subdir_to + filename_to, index=False)